In [223]:
import pandas as pd 
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pyproj
import pandas as pd
import regex as reg


Loading Data

In [224]:
# Load the data
mastra_df = pd.read_csv('mastra.csv')
aadt_df = pd.read_excel('AADT.xlsx')

In [225]:
# Load streetnetwork
streetnetwork_df = gpd.read_file('aalborg_network.gpkg')

Preperation

In [226]:
# Convert UTM32 to latitude and longitude (WGS 84)
utm_zone = 32  # UTM zone for your coordinates
proj_utm = pyproj.CRS(f'epsg:326{utm_zone}')  # EPSG code for UTM32N
proj_wgs = pyproj.CRS('epsg:4326')  # EPSG code for WGS 84

# Create a transformer object
transformer = pyproj.Transformer.from_crs(proj_utm, proj_wgs, always_xy=True)
# Function to convert UTM to WGS84
def utm_to_wgs(row):
    try:
        lon, lat = transformer.transform(row['Xkoordinat'], row['Ykoordinat'])
    except:
        lon, lat = transformer.transform(row['X-koordinat'], row['Y-koordinat'])
    return pd.Series({'latitude': lat, 'longitude': lon})

# Convert the coordinates
mastra_df[['latitude', 'longitude']] = mastra_df[['Xkoordinat', 'Ykoordinat']].apply(utm_to_wgs, axis=1)
mastra_df.drop(['Xkoordinat', 'Ykoordinat'], inplace=True, axis=1)
aadt_df[['latitude', 'longitude']] = aadt_df[['X-koordinat', 'Y-koordinat']].apply(utm_to_wgs, axis=1)
aadt_df.drop(['X-koordinat', 'Y-koordinat'], inplace=True, axis=1)

In [227]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 20

Clean data

In [228]:
# Verify the column names in the DataFrame
print(aadt_df.columns)

# reg ex to replace , with . on the whole document
aadt_df = aadt_df.replace({',': '.'}, regex=True)

# Correct column names if necessary
columns_to_convert = [
    'Lastbil %',
    '15% hast.-fraktil', 
    '85% hast.-fraktil', 
    '% over hastighedsgr.',
    '% o. hastgr + 10km/t', 
    '% o. hastgr + 20km/t',
    'Talte dage',
    'Gennemsnitshastighed',
    'Lastbil %',
    'Største time'
]

# Convert the specified columns to float64
aadt_df = aadt_df.astype({col: 'float64' for col in columns_to_convert})


# Correct column names if necessary
columns_to_convert = [
    'Vejdel',
    'Lokalitet',
    'Vejnavn',
    'Køretøjsart',
    'StedId'
]

# Convert the specified columns to float64
aadt_df = aadt_df.astype({col: 'string' for col in columns_to_convert})

Index(['Dataejer', 'Bestyrer', 'Adm.vejnr', 'Vejdel', 'Kilometer', 'Meter',
       'Lokalitet', 'Vejnavn', 'Køretøjsart', 'År', 'StedId', 'Talte dage',
       'ÅDT', 'HDT', 'JDT', 'HDT.1', 'Gennemsnitshastighed', 'Æ10 Høj',
       'Æ10 Lav', 'Lastbil ÅDT', 'o. 1250 ÅDT', 'Lastbil %',
       '15% hast.-fraktil', '85% hast.-fraktil', '% over hastighedsgr.',
       '% o. hastgr + 10km/t', '% o. hastgr + 20km/t', 'Hastighedsgrænse',
       'Største time', 'Største time datotid', 'Morgen spids time',
       'Morgen spids st. tid', 'Efterm. spids time', 'Efterm. spids st. t.',
       'latitude', 'longitude'],
      dtype='object')


In [229]:
aadt_df['Største time datotid'] = pd.to_datetime(aadt_df['Største time datotid'], errors='coerce')
aadt_df['Morgen spids st. tid'] = aadt_df['Morgen spids st. tid'].replace(':', np.nan)
aadt_df['Morgen spids st. tid (since Midnight)'] = pd.to_timedelta(aadt_df['Morgen spids st. tid'] + ':00', errors='coerce')
aadt_df.drop('Morgen spids st. tid', axis=1, inplace=True)


aadt_df['Efterm. spids st. t.'] = aadt_df['Efterm. spids st. t.'].replace(':', np.nan)
aadt_df['Efterm. spids st. t. (since Midnight)'] = pd.to_timedelta(aadt_df['Efterm. spids st. t.'] + ':00', errors='coerce')
aadt_df.drop('Efterm. spids st. t.', axis=1, inplace=True)

C:\Users\gonde\AppData\Local\Temp\ipykernel_12040\4129793633.py:1: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  aadt_df['Største time datotid'] = pd.to_datetime(aadt_df['Største time datotid'], errors='coerce')


mastra

In [230]:
# Verify the column names in the DataFrame
print(mastra_df.columns)

# reg ex to replace , with . on the whole document
mastra_df = mastra_df.replace({',': '.'}, regex=True)
mastra_df = mastra_df.replace({'LÃ¸rdag': "Loerdag"}, regex=True)
mastra_df = mastra_df.replace({'SÃ¸ndag': "Soendag"}, regex=True)


mastra_df.rename(columns={{'MÃ¥ned': 'Maened'}})
mastra_df.rename(columns={'Ãrstal': 'Aarstal'}, inplace=True)
mastra_df.rename(columns={'ï»¿Vej-Id': 'Vej-Id'}, inplace=True)

mastra_df.drop(columns=['Unnamed: 1', 'Unnamed: 2'], inplace=True)

# Correct column names if necessary
columns_to_convert = [

]

# Convert the specified columns to float64
mastra_df = mastra_df.astype({col: 'float64' for col in columns_to_convert})


# Correct column names if necessary
columns_to_convert = [
    'Trafiktype',
    'Dagtype',
    'Vej-Id',
    'Retning',
    'Trafiktype'
]

# Convert the specified columns to float64
mastra_df = mastra_df.astype({col: 'string' for col in columns_to_convert})

mastra_df = mastra_df.replace({'<NA>': ""}, regex=True)

Index(['ï»¿Vej-Id', 'Unnamed: 1', 'Unnamed: 2', 'Kilometer', 'Retning', 'Spor',
       'Dato', 'Ãrstal', 'MÃ¥ned', 'Dag',
       ...
       '22:00-22:15', '22:15-22:30', '22:30-22:45', '22:45-23:00',
       '23:00-23:15', '23:15-23:30', '23:30-23:45', '23:45-24:00', 'latitude',
       'longitude'],
      dtype='object', length=110)


In [231]:
pd.options.display.max_rows = 5
mastra_df

,Vej-Id,Unnamed: 1,Unnamed: 2,Kilometer,Retning,Spor,Dato,Aarstal,Maened,Dag,Dagtype,Trafiktype,00:00-00:15,00:15-00:30,00:30-00:45,00:45-01:00,01:00-01:15,01:15-01:30,01:30-01:45,01:45-02:00,02:00-02:15,02:15-02:30,02:30-02:45,02:45-03:00,03:00-03:15,03:15-03:30,03:30-03:45,03:45-04:00,04:00-04:15,04:15-04:30,04:30-04:45,04:45-05:00,05:00-05:15,05:15-05:30,05:30-05:45,05:45-06:00,06:00-06:15,06:15-06:30,06:30-06:45,06:45-07:00,07:00-07:15,07:15-07:30,07:30-07:45,07:45-08:00,08:00-08:15,08:15-08:30,08:30-08:45,08:45-09:00,09:00-09:15,09:15-09:30,09:30-09:45,09:45-10:00,10:00-10:15,10:15-10:30,10:30-10:45,10:45-11:00,11:00-11:15,11:15-11:30,11:30-11:45,11:45-12:00,12:00-12:15,12:15-12:30,12:30-12:45,12:45-13:00,13:00-13:15,13:15-13:30,13:30-13:45,13:45-14:00,14:00-14:15,14:15-14:30,14:30-14:45,14:45-15:00,15:00-15:15,15:15-15:30,15:30-15:45,15:45-16:00,16:00-16:15,16:15-16:30,16:30-16:45,16:45-17:00,17:00-17:15,17:15-17:30,17:30-17:45,17:45-18:00,18:00-18:15,18:15-18:30,18:30-18:45,18:45-19:00,19:00-19:15,19:15-19:30,19:30-19:45,19:45-20:00,20:00-20:15,20:15-20:30,20:30-20:45,20:45-21:00,21:00-21:15,21:15-21:30,21:30-21:45,21:45-22:00,22:00-22:15,22:15-22:30,22:30-22:45,22:45-23:00,23:00-23:15,23:15-23:30,23:30-23:45,23:45-24:00,latitude,longitude
0,0 70-0 277/ 100 -,NaN,NaN,277.0,-,NaN,01.01.2023,2023.0,1.0,1.0,Helligdag,BLANDET,11.0,20.0,41.0,88.0,103.0,111.0,99.0,92.0,87.0,76.0,74.0,69.0,64.0,70.0,45.0,42.0,42.0,26.0,23.0,27.0,16.0,12.0,12.0,14.0,20.0,26.0,26.0,27.0,22.0,33.0,27.0,22.0,31.0,26.0,25.0,40.0,47.0,67.0,69.0,78.0,123.0,173.0,221.0,230.0,295.0,434.0,470.0,475.0,445.0,487.0,481.0,511.0,517.0,523.0,473.0,443.0,431.0,432.0,402.0,374.0,375.0,381.0,317.0,331.0,289.0,296.0,298.0,257.0,265.0,289.0,196.0,182.0,174.0,158.0,139.0,146.0,126.0,126.0,115.0,161.0,169.0,144.0,230.0,99.0,86.0,58.0,49.0,45.0,191.0,182.0,57.0,34.0,NaN,NaN,18.0,NaN,56.942790,9.859120
1,0 70-0 277/ 100 -,NaN,NaN,277.0,-,NaN,02.01.2023,2023.0,1.0,2.0,Hverdag,BLANDET,14.0,12.0,15.0,13.0,12.0,10.0,24.0,25.0,9.0,12.0,15.0,13.0,14.0,14.0,20.0,25.0,38.0,44.0,53.0,84.0,89.0,131.0,169.0,190.0,254.0,377.0,405.0,490.0,383.0,511.0,473.0,427.0,303.0,271.0,249.0,245.0,227.0,264.0,231.0,234.0,245.0,352.0,337.0,351.0,340.0,307.0,316.0,340.0,454.0,387.0,456.0,331.0,346.0,319.0,356.0,354.0,353.0,383.0,425.0,424.0,501.0,641.0,691.0,635.0,563.0,618.0,482.0,411.0,247.0,259.0,250.0,214.0,161.0,176.0,151.0,125.0,103.0,130.0,140.0,217.0,171.0,128.0,259.0,114.0,98.0,91.0,70.0,59.0,172.0,149.0,64.0,49.0,NaN,NaN,41.0,NaN,56.942790,9.859120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269040,851 8519680-0 0/ 470 T,NaN,NaN,0.0,T,NaN,31.08.2024,2024.0,8.0,31.0,Loerdag,BO-ARB,60.0,61.0,60.0,63.0,48.0,55.0,42.0,37.0,26.0,30.0,25.0,33.0,29.0,20.0,19.0,19.0,14.0,25.0,14.0,14.0,17.0,19.0,28.0,31.0,21.0,41.0,95.0,68.0,61.0,70.0,75.0,91.0,75.0,76.0,139.0,142.0,180.0,190.0,224.0,182.0,225.0,225.0,244.0,255.0,226.0,222.0,262.0,250.0,260.0,252.0,245.0,237.0,228.0,231.0,241.0,216.0,229.0,256.0,245.0,206.0,224.0,216.0,223.0,201.0,220.0,181.0,214.0,201.0,256.0,180.0,201.0,196.0,192.0,190.0,167.0,166.0,168.0,142.0,147.0,163.0,131.0,140.0,147.0,135.0,125.0,151.0,104.0,99.0,84.0,90.0,77.0,90.0,101.0,77.0,81.0,87.0,57.060566,9.933534
269041,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [232]:
# show dtype of every column
pd.options.display.max_rows = None
print(mastra_df.dtypes)


Vej-Id         string[python]
Unnamed: 1            float64
Unnamed: 2            float64
Kilometer             float64
Retning        string[python]
Spor                  float64
Dato                   object
Aarstal               float64
Maened                float64
Dag                   float64
Dagtype        string[python]
Trafiktype     string[python]
00:00-00:15           float64
00:15-00:30           float64
00:30-00:45           float64
00:45-01:00           float64
01:00-01:15           float64
01:15-01:30           float64
01:30-01:45           float64
01:45-02:00           float64
02:00-02:15           float64
02:15-02:30           float64
02:30-02:45           float64
02:45-03:00           float64
03:00-03:15           float64
03:15-03:30           float64
03:30-03:45           float64
03:45-04:00           float64
04:00-04:15           float64
04:15-04:30           float64
04:30-04:45           float64
04:45-05:00           float64
05:00-05:15           float64
05:15-05:3